In [ ]:
%matplotlib inline
import pandas as pd
import os
import random
import re
import nltk
import string
import urllib2
import math
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
# from gensim import corpora
# from gensim.models import LdaMulticore
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import copy as cp
from bs4 import BeautifulSoup
from functools import reduce 
import operator
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import datetime
from textblob import TextBlob

In [ ]:
#2013-2016 emergence of populist concerns 
# Frequency of Populist key words 
data_file = '~/Documents/Small Projects/AAPOR_2018/Populism/'
populist_posts = pd.read_csv(data_file + "populist_comments_sample.csv")

In [3]:
populist_posts['year'] = populist_posts['time_created'].apply(lambda x :datetime.datetime.fromtimestamp(x).year)
populist_posts['month'] = populist_posts['time_created'].apply(lambda x :datetime.datetime.fromtimestamp(x).month)

In [4]:
#data = populist_posts #just rename to make it faster

In [5]:
train_topic_model = shuffle(populist_posts)[:20000]
data = train_topic_model
#we only train on 20,000 entries because it otherwise takes soooooo long

In [ ]:
#make a subclass of the vectorizer that lemmatizes so we don't end up with wierd truncations
# lemm = WordNetLemmatizer()
# class LemmaCountVectorizer(CountVectorizer):
#     def build_analyzer(self):
#         analyzer = super(LemmaCountVectorizer, self).build_analyzer()
#         return lambda doc: (lemm.lemmatize(w) for w in analyzer(doc))

In [7]:
all_text = list(data['body'])

### Get the Top Words used by Subreddit and Time

In [10]:
def get_top_words(text):
    tf_vectorizer = CountVectorizer(max_df=0.95, 
                                         min_df=len(text)/100,
                                         stop_words='english',
                                         decode_error='ignore')
    tf = tf_vectorizer.fit_transform(text)
    feature_names = tf_vectorizer.get_feature_names()
    count_vec = np.asarray(tf.sum(axis=0)).ravel()
    zipped = list(zip(feature_names, count_vec))
    words, freqs = (list(x) for x in zip(*sorted(zipped, key=lambda x: x[1], reverse=True)))
    words = words[:20]
    freqs = freqs[:20]
    return words
#     top_words = pd.DataFrame([words, freqs]).T
#     top_words

In [11]:
#the top words
get_top_words(list(data['body']))

[u'trump',
 u'people',
 u'gt',
 u'just',
 u'like',
 u'don',
 u'clinton',
 u'white',
 u'think',
 u'com',
 u'www',
 u'right',
 u'http',
 u'know',
 u'hillary',
 u'going',
 u'said',
 u'https',
 u'politics',
 u'make']

In [12]:
list_of_subreddits = ['worldnews','news','history','politics', 'democrats'] #these are the largest
top_words_grid = []
cols = []
for subreddit in list_of_subreddits:
    cols.append(subreddit)
    top_words_grid.append( get_top_words(list(data[data['subreddit']==subreddit]['body'])))
    print subreddit
top_words_df = pd.DataFrame(top_words_grid).T
top_words_df.columns = cols
top_words_df.head()

worldnews
news
history
politics
democrats


,worldnews,news,history,politics,democrats
0,trump,people,amp,trump,trump
1,people,white,people,people,people
2,gt,trump,war,clinton,clinton
3,just,just,white,gt,sanders
4,like,like,like,just,like


In [13]:
top_words_grid_yr = []
year_list = []
for year in [2013, 2014, 2015, 2016, 2017]:
    year_list.append(year)
    top_words_grid_yr.append(get_top_words(list(data[data['year']==year]['body'])))
    print year
top_words_df_year = pd.DataFrame(top_words_grid_yr).T
top_words_df_year.columns = year_list
top_words_df_year.head()

2013
2014
2015
2016
2017


,2013,2014,2015,2016,2017
0,people,people,people,trump,trump
1,gt,white,gt,people,people
2,white,just,just,clinton,gt
3,just,like,white,gt,just
4,like,gt,like,just,like


### Topic Modelling

In [16]:
tf_vectorizer = CountVectorizer(max_df=0.95, 
                                         min_df=len(all_text)/100,
                                         stop_words='english',
                                         decode_error='ignore')
transformed_text = tf_vectorizer.fit_transform(all_text)

In [17]:
topic_model = LatentDirichletAllocation(n_components=15, max_iter=5,
                                learning_method = 'online',
                                random_state = 0)

topic_model.fit(transformed_text)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_components=15, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [18]:
feature_names = tf_vectorizer.get_feature_names()
for topic in topic_model.components_:
    word_ixes = topic.argsort()[:-10:-1]
    words = [feature_names[i] for i in word_ixes]
    print words
    print "______________________________"

[u'trump', u'vote', u'party', u'supporters', u'election', u'republicans', u'win', u'republican', u'candidate']
______________________________
[u'look', u'don', u'yes', u'want', u'wasn', u'liberal', u'jobs', u'trumps', u'actual']
______________________________
[u'trump', u'just', u'like', u'doesn', u'thing', u'doing', u'right', u'know', u'media']
______________________________
[u'obama', u'war', u'didn', u'years', u'trump', u'wouldn', u'tell', u'help', u'office']
______________________________
[u'gt', u'right', u'law', u'illegal', u'immigrants', u'rights', u'police', u'religion', u'example']
______________________________
[u'politics', u'reddit', u'amp', u'com', u'message', u'https', u'comments', u'www', u'removal']
______________________________
[u'public', u'muslims', u'old', u'countries', u'immigration', u'eu', u'attack', u'dnc', u'fact']
______________________________
[u'com', u'http', u'www', u'https', u'2016', u'news', u'org', u'gt', u'article']
______________________________
[u't

In [19]:
def which_topic(topic_model, text):
    '''
    Helper function to return a random topic if the best match is tied
    '''
    x = topic_model.transform(tf_vectorizer.transform([text]))[0]
    max_value = x[x.argsort()[-1]]
    possible_topics = []
    for index, value in enumerate(x):
        if value == max_value:
            possible_topics.append(index)
    return np.random.choice(possible_topics, 1)[0]

In [20]:
data['topic'] = data['body'].apply(lambda x: which_topic(topic_model,x))

### Sentiment

In [21]:
#try excepts are to skip unicode errors which I don't want to deal with
#Somewhat slow on partial data (~30 sec)
def get_polarity(text):
    try:
        return TextBlob(text).sentiment.polarity
    except:
        return np.nan
def get_subjectivity(text):
    try:
        return TextBlob(text).sentiment.subjectivity
    except:
        return np.nan
data['polarity'] = data['body'].apply(lambda x: get_polarity(x))
data['subjectivity'] = data['body'].apply(lambda x: get_subjectivity(x))

### Clean Up and Analysis

In [22]:
#topic balance over time, I'm imagining one of those area plots
topics_over_time = data.groupby(['year', 'month', 'topic']).agg({'body': 'count'}).reset_index().rename(columns = {'body':'count'})
topics_over_time.head()

,year,month,topic,count
0,2012,12,12,1
1,2013,1,1,1
2,2013,1,2,5
3,2013,1,3,1
4,2013,1,4,4


In [23]:
sentiment_over_time = data.groupby(['year', 'month']).agg({'polarity': np.mean, 'subjectivity': np.mean})
#use this to make a  line graph, show that polarity isn't hugely changing ovet time neither is subjectivity
sentiment_over_time.head()

polarity  subjectivity
year month                        
2012 12    -0.242857      0.467262
2013 1      0.025298      0.454223
     2      0.064368      0.392500
     3      0.043157      0.415356
     4      0.036377      0.379031

In [24]:
sentiment_by_topic = data.groupby('topic').agg({'subjectivity': np.mean, 'polarity': np.mean, })
#also not big differences by topic
sentiment_by_topic.head()

,polarity,subjectivity
topic,,
0,0.046731,0.474978
1,0.038213,0.436526
2,0.052158,0.475796
3,0.064343,0.457058
4,0.031493,0.444088


In [25]:
data['Clinton'] = data['body'].apply(lambda x : True if 'clinton' in x.lower() or 'hilary' in x.lower() else False)
data['Trump'] = data['body'].apply(lambda x : True if 'trump' in x.lower() or 'donald' in x.lower() else False)

In [26]:
clinton_sentiment = np.mean(data[data['Clinton']]['polarity'])
trump_sentiment = np.mean(data[data['Trump']]['polarity'])
clinton_sentiment, trump_sentiment

(0.06337006754296344, 0.04930149277822336)

In [27]:
clinton_over_time = data[data['Clinton']].groupby(['year', 'month']).agg({'polarity': np.mean, 'subjectivity': np.mean, 'body': 'count'}).rename(columns= {'body': 'post_count'})
clinton_over_time.head()

polarity  post_count  subjectivity
year month                                    
2013 1     -0.197586           4      0.475020
     2      0.086819           9      0.352403
     3      0.018503           3      0.513577
     4     -0.025568           1      0.477273
     5      0.060191           2      0.456490

In [28]:
trump_over_time = data[data['Trump']].groupby(['year', 'month']).agg({'polarity': np.mean, 'subjectivity': np.mean, 'body': 'count'}).rename(columns= {'body': 'post_count'})
trump_over_time.head()

polarity  post_count  subjectivity
year month                                    
2013 1     -0.005309           3      0.687811
     3      0.150000           1      0.650000
     4      0.000000           1      0.500000
     5      0.248398           2      0.485931
     6      0.400000           2      0.775000

In [29]:
data.groupby('subreddit')['body'].count() #not enough in a lot of subreddits for much divided analysis

subreddit
business               14
democrats              48
education               2
energy                  5
environment            22
feminisms               6
history                70
law                    26
moderatepolitics       16
news                 2369
politics            13012
progressive            24
racism                  8
socialism             113
ukpolitics            685
uspolitics             10
worldevents             5
worldnews            3457
worldpolitics         108
Name: body, dtype: int64

In [ ]:
# (optional) Word Embedding

In [ ]:
# Sentiment Analysis 